# Step 1 Python Dependence

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from itertools import product
from gridWalk import bicycleGridRiding
from DPforGridBike_v1 import ValueIteration

Python Settings

In [ ]:
# 设置 numpy 科学计数法
np.set_printoptions(suppress=True, threshold=5000)
# 设置 pandas 的格式
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.float_format', lambda x: '%.8f' % x)
# Full Tensor
torch.set_printoptions(sci_mode=False, profile="full")
torch.autograd.set_detect_anomaly(True)

### 设置随机数种子

In [ ]:
import random

# Step 2 Functions

In [ ]:
class TrajProccess():
    def __init__(self, env, trajData, device):
        self.trajData = trajData
        self.env = env
        self.device = device

    @ property
    def maxTrajLength(self):
        '''
        Max Length For Given Trajectories
        :param trajData: Dataframe, Given Trajectories
        '''
        trajIDs = np.unique(self.trajData['TrajID'])

        trajectoryList = []
        trajectoryLength = []
        for trajID in trajIDs:
            trajectory = []
            trajID_Data = self.trajData[self.trajData['TrajID'] == trajID].copy()  # The Information of Current Trajectory

            # [state, action, nextState]
            trajID_Data.apply(lambda x: trajectory.append([x['state'], x['action'], x['nextState']]), axis=1)

            trajectoryList.append(trajectory)  # Current Trajectory
            trajectoryLength.append(len(trajectory))

        return trajectoryList, max(trajectoryLength)

    @property
    def Padding(self):
        '''
        Trajectories Padding Function
        :param trajectories:
        :param maxTL: Max Length Of Given Trajectories
        :return: trajMatrix, size = [numTraj, maxTL]
        '''
        trajectories = self.maxTrajLength[0]
        maxLength = self.maxTrajLength[1]

        trajectoryMatrix = []
        for trajectory in trajectories:
            if len(trajectory) != maxLength:
                for i in range((maxLength - len(trajectory))):
                    trajectory.append([-1, -1, -1])
                trajectoryMatrix.append(trajectory)
            else:
                trajectoryMatrix.append(trajectory)

        return torch.tensor(trajectoryMatrix, dtype=torch.float32).to(self.device)

    def find_svf(self):
        # print('Finding SVF ···')
        numStates = len(self.env.states)    # numStates
        # trajectoryList = self.Padding()    # []
        trajectoryList = self.maxTrajLength[0]    # []
        svf = np.zeros(numStates)  # [numStates, 1]

        for trajectory in trajectoryList:
            # print('轨迹为{}'.format(trajectory))
            for stateActionPair in trajectory:
                # [1373.0, 0.0, 1373.0]
                state = int(stateActionPair[0])
                svf[state] += 1
                # if state != -1:
                #     svf[state] += 1

        # svf /= trajectories.shape[0]
        # [[s,a], [s,a],[s,a]],
        # [[s,a], [s,a],[s,a]]
        svf /= len(trajectoryList)

        a = torch.tensor(svf, dtype=torch.float32).to(self.device)
        a1 = F.normalize(a, dim=0)
        
        return a1

    def find_expected_svf(self, r=None):
        # print('Finding Expected SVF ······')
        env = self.env

        # trajMatrix.shape -> [21237, 120, 3]
        trajMatrix = self.Padding    # [numTraj, maxLength, 3]
        n_trajectories = trajMatrix.shape[0]  # [numTrajectories, 1]
        trajectory_length = trajMatrix.shape[1]  # [trajectoryLength, 1]

        # ########### 修改了此处传参的过程 ###########
        # if r != None:
        #     agent.rewardFunc = r.cpu().detach().numpy()    # [numStates, 1]
        # else:
        #     agent.rewardFunc = [env.rewardFunc(agentloc) for agentloc in range(len(env.states))]    # 此处传参的过程可能存在问题
            
        ######## 5月11日修改结果  ######
        if r != None:
            r1 = r.cpu().detach().numpy()    # [numStates, 1]
        else:
            r1 = [env.rewardFunc(agentloc) for agentloc in range(len(env.states))]    # 此处传参的过程可能存在问题

        agent = ValueIteration(env, r1)
        GridV = agent.plan(threshold=0.01)
        policy = agent.get_policy(GridV)
        # Reward Function Is Not Available - Why ?    -> 没有将奖励函数传入至 Value Iteration 之中

        start_state_count = torch.zeros(len(env.states), dtype=torch.float32)  # [numStates, 1]
        for trajectory in trajMatrix:
            start_state_count[int(trajectory[0, 0])] += 1
        p_start_state = start_state_count / n_trajectories  # the Probability of Start State

        expected_svf = torch.tile(p_start_state, (trajectory_length, 1)).T    # [trajLength, numStates] -> [numStates, trajLength]
        for t in range(1, trajectory_length):
            expected_svf[:, t] = 0  # Trajectory 长度为 trajLength, 初始化为起始点出现的概率
            for i, j in product(range(len(env.states)), range(env.action_space.n)):
                action = env.index2Action(j)
                transitionProb = env.transitFunc(i, action)    # dict

                # Next State
                for k in transitionProb.keys():
                    expected_svf[k, t] += (expected_svf[i, t - 1] *
                                           policy[i, j] *  # Stochastic policy
                                           transitionProb[k])
            print('第{}个位置已经完成'.format(t))
            
        b = expected_svf.sum(dim=1, dtype=torch.float32).to(self.device)  # [numStates, 1]
        b1 = F.normalize(b, dim=0)
        
        return b1

图卷积得到的 Reward 传入矩阵之中，不存在道路的 reward 强制修改为某个特定的常数，以得到智能体环境的高效表示，避免噪音的影响；
- 这种情况下卷积神经网络能否得到相似的结果？
- 首先尝试卷积神经网络作为依赖学习的方式
- 似乎不需要将图卷积神经网络加入到函数的设计之中

更新日志
- 2024.5.18 9：05 按照上述的思路，在不加入卷积神经网络或者是图神经网络的基础上，学习其奖励以明确这种方式是否可以收敛 

In [ ]:
class deepIRL():
    def __init__(self, env, stateFeatures, trajProcess,
                 ini_net, discount, device, epochs, passList,
                 learning_rate=0.1, l1=0.1, l2=0.1):
        self.env = env
        self.stateFeatures = stateFeatures
        self.passList = passList
        self.trajProcess = trajProcess
        self.ini_net = ini_net
        self.discount = discount
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l1 = l1
        self.l2 = l2
        self.device = device

    def update(self, net, alpha):
        # Forward Propagation
        # print('Forward Propagation ····· ')
        n_states, d_states = self.stateFeatures.shape  # numStates, dimension
        reward = torch.matmul(net(self.stateFeatures), alpha).reshape(n_states,).to(self.device)    # [numStates, dimensions]
        reward = (reward - reward.mean()) / reward.std()    # Reward Standardization

        # 2024.5.18 日晚更新：路径规划器仍然需要获得所有的奖励
        # 加入一个变量为 reward_temp 保存暂时的奖励
        reward_temp = torch.zeros(len(self.env.states))
        # passList 应该为一个可以通过的 FID 列表
        # rewardTemp 应该是一个与环境大小相同的网格单元
        i = 0
        for j in self.passList:
            reward_temp[j] = reward[i]
            i += 1
        
        # print(reward)
        weights = []
        biases = []
        for name, param in net.named_parameters():
            if 'weight' in name:
                weights.append(param)
            elif 'bias' in name:
                biases.append(param)

        # print('Back Propagation ······')
        adagrad_epsilon = 1e-6  # AdaGrad numerical stability.
        expected_svf = trajProcess.find_expected_svf(r=reward_temp).reshape(len(self.env.states),)  # torch.tensor, size=[numStates, 1] -> After Normalization
        svf = trajProcess.find_svf().reshape(len(self.env.states),)  # torch.tensor, size=[numStates, 1] -> After Normalization
        grad = svf - expected_svf  # [numStates, 1]
        grad1 = torch.Tensor([grad[s] for s in self.passList]).to(self.device)    # n_states

        updates = []
        hist_alpha_grad = torch.zeros(alpha.shape).to(self.device)  # [dimensions, 1]
        output = net(self.stateFeatures)    # [numStates, output_dim] -> [numStates, dimension] -> [numStates, 4]
        alpha_grad = (torch.matmul(grad1.T, output).
                      reshape(alpha.shape))  # [output_dim, 1] -> [dimensions, 1] -> [4, 1]
        hist_alpha_grad += alpha_grad ** 2  # history grad    [output_dim, 1] -> [dimensions, 1]
        adj_alpha_grad = alpha_grad / (adagrad_epsilon + torch.sqrt(hist_alpha_grad))
        updates.append((alpha, alpha + adj_alpha_grad * self.learning_rate))


        def grad_for_state(s, theta, svf_diff, r):
            """
            Calculate the gradient with respect to theta for one state.
            """
            regularisation = torch.sum(torch.abs(theta)) * self.l1 + torch.sum(theta ** 2) * self.l2
            autograd = torch.autograd.grad(r[s], theta, retain_graph=True)[0]
            svf_diff_s = svf_diff[s]
            return  svf_diff_s * autograd - regularisation
            

        hist_w_grads = [torch.zeros_like(weight).to(self.device) for weight in weights]
        for i, W in enumerate(weights):
            w_gradList = []
            for state in range(n_states):
                w_template_grad = grad_for_state(state, W, grad1, reward)
                w_gradList.append(w_template_grad)
            
            # 计算梯度
            w_grads = torch.stack(w_gradList)
            w_grad = torch.sum(w_grads, dim=0)
            # 清零历史梯度
            if hist_w_grads[i] is not None:
                hist_w_grads[i].zero_()
            # 更新历史梯度平方累积和
            hist_w_grads[i] += w_grad ** 2
            # 计算调整后的梯度
            adj_w_grad = w_grad / (adagrad_epsilon + torch.sqrt(hist_w_grads[i]))
            # 添加参数更新规则到更新列表
            updates.append((W, W + adj_w_grad * self.learning_rate))

        hist_b_grads = [torch.zeros_like(bias).to(self.device) for bias in biases]
        for i, b in enumerate(biases):
            # 计算梯度
            b_gradList = []
            for state in range(n_states):
                b_gradList.append(grad_for_state(state, b, grad1, reward))
            # 计算梯度
            b_grads = torch.stack(b_gradList)
            b_grad = torch.sum(b_grads, dim=0)
            # 清零历史梯度
            if hist_b_grads[i] is not None:
                hist_b_grads[i].zero_()
            # 更新历史梯度平方累积和
            hist_b_grads[i] += b_grad ** 2
            # 计算调整后的梯度
            adj_b_grad = b_grad / (adagrad_epsilon + torch.sqrt(hist_b_grads[i]))
            # 添加参数更新规则到更新列表
            updates.append((b, b + adj_b_grad * self.learning_rate))

        n = int((len(updates) - 1) / 2)
        with torch.no_grad():
            for name, param in net[0].named_parameters():
                if 'weight' in name:
                    param.data = updates[1][1]
                elif 'bias' in name:
                    param.data = updates[n+1][1]
            
            for name, param in net[1].named_parameters():
                if 'weight' in name:
                    param.data = updates[2][1]
                elif 'bias' in name:
                    param.data = updates[n+2][1]
                    
        net.zero_grad()
        rewardNew = torch.matmul(net(self.stateFeatures), updates[0][1]).reshape(n_states,)

        # 2024.5.19 日更新：对 reward 进行标准化操作
        rewardNew1 = (rewardNew.max()-rewardNew)/(rewardNew.max()-rewardNew.min())
        
        # ### 2024.5.18 9:25 此处更新，将所有不存在路段的网格奖励归 0
        # rewardNew1 = torch.zeros_like(rewardNew)
        # for loc in self.PassList:
        #     rewardNew1[loc] = rewardNew[loc]
            
        return updates[0][1], net, rewardNew1

    def train(self, n):
        ini_net = self.ini_net
        ############
        ini_alpha = torch.normal(mean=0, std=0.01, size=(4, 1)).to(self.device)    # 此处的代码也需要适应神经网络的变化
        # rewardList= []
        # svfDiff = []
        rewardList = []
        stdList = []
        num_epochs = self.epochs
        for i in range(n):
            with tqdm(total=int(num_epochs / n), desc='Iteration %d' % i) as pbar:
                for i_epoch in range(int(num_epochs / n)):
                    # 奖励的更新
                    alpha, output_net, reward = self.update(net=ini_net, alpha=ini_alpha)
                    ini_net = nn.Sequential(*list(output_net.children()))
                    ini_alpha = alpha
                    # rewardList.append(reward)
                    # svfDiff.append(svf)
                    rewardList.append(reward.mean())
                    stdList.append(reward.std())

                    if (i_epoch + 1) % n == 0:
                        pbar.set_postfix({
                            'epoch':
                                '%d' % (num_epochs / n * i + i_epoch + 1),
                            # 'return':
                            # '%.3f' % np.mean(return_list[-10:])
                        })
                    pbar.update(1)
                    
        return rewardList, stdList, reward, ini_alpha

# Step 3 创建智能体运行的环境并训练模型

In [ ]:
print('The Script Is Running······')

In [ ]:
attrTable = pd.read_csv(r'./03_stateAttrNorm_output.txt', sep=',')
attrTable1 = pd.read_csv(r'./NormAttrPass.csv', sep=',')

In [ ]:
os.chdir(r'F:\BaiduNetdiskDownload\共享单车轨迹\共享单车轨迹\01_研究生毕业论文\1_2_DataPreProcess')
trajFile = r'SAPairCom.txt'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trajData = pd.read_csv(trajFile)  # ['FID', 'OrderID', 'TrajID', 'timeStamp', 'LG_ID', 'state', 'action', 'nextState']

# Create Environment For Bicycle To Move
env = bicycleGridRiding(attrTable=attrTable)
state, _ = env.reset()
print('Environment Is Successfully Built')

trajProcess = TrajProccess(env=env, trajData=trajData, device=device)

In [ ]:
passListIndex = attrTable1['FID'].values

In [ ]:
# Inverse Reinforcement Learning
# Training
stateFeatures = torch.tensor(np.vstack([attrTable1[attrTable1['FID']==s].to_numpy()[0, 2:] for s in passListIndex]),
                                 dtype=torch.float32).to(device)
n_states, d_states = stateFeatures.shape    # numStates, dimension

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
setup_seed(2024)

In [ ]:
# Initialize Model Parameters
net = nn.Sequential(nn.Linear(23, 12),
                    nn.Linear(12, 6),
                    nn.BatchNorm1d(6),
                    nn.Softmax(),
                    nn.Linear(6, 4),
                    nn.BatchNorm1d(4),
                    nn.Softmax())
def init_weights(m):
    '''
    :param m: Neural Network Module
    :return:
    '''
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

net.apply(init_weights)
net.to(device=device)

In [ ]:
n_states, d_states
ini_alpha = torch.normal(mean=0, std=0.01, size=(4, 1)).to(device)

In [ ]:
ini_net = nn.Sequential(*list(net.children()))

agent = deepIRL(env=env, 
                stateFeatures=stateFeatures, 
                trajProcess=trajProcess, 
                ini_net=ini_net, 
                discount=0.9, 
                device=device, 
                epochs=18,
                passList=passListIndex)
rewardList, stdList, reward, ini_alpha = agent.train(n=18)

In [ ]:
rewardList

In [ ]:
stdList

In [ ]:
reward

保存运算所得的结果以及其参数

In [ ]:
ini_alpha1 = ini_alpha.cpu().detach().numpy()
np.save('Alpha_pass_Final', ini_alpha1)

In [ ]:
torch.save(net, './NetParam_Pass_Final.pkl')

In [ ]:
reward_cpu = reward.cpu().detach().numpy()
dataFrame = pd.DataFrame({'index':passListIndex, 'reward':reward_cpu})
dataFrame.to_csv(r'./reward_Pass_Final.csv', encoding='utf-8')

In [ ]:
# reward_List_cpu = rewardList.cpu().detach().numpy()
# stdList_cpu = stdList.cpu().detach().numpy()
df1 = pd.DataFrame({'index':[i for i in range(len(rewardList))], 
                    'reward':[rewardMean.cpu().detach().numpy() for rewardMean in rewardList], 
                    'std': [stdMean.cpu().detach().numpy() for stdMean in stdList]})

In [ ]:
df1

In [ ]:
df1.to_csv(r'./rewardTrend.csv', encoding='utf-8')